In [1]:
import pandas as pd
import sqlite3
import os
import requests
import json

In [2]:
master_df = pd.read_excel("decoded_with_sources.xlsx")

In [3]:
collected_rows = []

for _, row in master_df.iterrows():
    slug = row['slug']
    source = row['source_file']
    if not os.path.exists(source):
        print(f"Source file not found: {source}")
        continue
    
    try:
        conn = sqlite3.connect(source)
        query = "SELECT * FROM sampled_collections WHERE slug_name = ?"
        result = pd.read_sql(query, conn, params=(slug,))
        conn.close()
        
        if not result.empty:
            collected_rows.append(result)
        else:
            print(f"No data found for slug '{slug}' in {source}")
    except Exception as e:
        print(f"Error reading from {source} for slug {slug}: {e}")

# Combine all rows into one DataFrame
if collected_rows:
    combined_df = pd.concat(collected_rows, ignore_index=True)
    
    # Write to new SQLite DB
    output_db = "sampling_frame.db"
    conn_out = sqlite3.connect(output_db)
    combined_df.to_sql("sampled_collections", conn_out, index=False, if_exists="replace")
    conn_out.close()
    
    print(f"✅ Combined data written to '{output_db}' with {len(combined_df)} rows.")
else:
    print("⚠️ No data collected. Check slug names or DB file paths.")

✅ Combined data written to 'sampling_frame.db' with 258 rows.


C:\Users\Sören Schlißke\AppData\Local\Temp\ipykernel_24900\1538753107.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(collected_rows, ignore_index=True)


In [4]:
def create_opensea_nfts_table(db_path="sampling_frame.db", table_name="opensea_nfts"):
    """
    Creates a table with columns for the relevant fields from the OpenSea NFT JSON.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Create table if not exists
    c.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        
        -- Linking field from 'sampled_collections' table
        slug_name TEXT,
        
        -- Fields from the NFT JSON:
        identifier TEXT,
        contract TEXT,
        token_standard TEXT,
        name TEXT,
        description TEXT,
        image_url TEXT,
        display_image_url TEXT,
        display_animation_url TEXT,
        metadata_url TEXT,
        opensea_url TEXT,
        updated_at TEXT,
        is_disabled INTEGER,
        is_nsfw INTEGER,
        
        fetched_at TEXT
    )
    """)
    
    conn.commit()
    conn.close()

def fetch_opensea_nfts(collection_slug, api_key, limit=10):
    """
    Calls the OpenSea endpoint:
      GET /api/v2/collection/{collection_slug}/nfts?limit={limit}
    Returns the parsed JSON dict with:
       { "nfts": [ { ... }, {...} ], "next": "string" }
    If there's an error, returns an empty dict.
    """
    base_url = "https://api.opensea.io/api/v2/collection"
    url = f"{base_url}/{collection_slug}/nfts"
    
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {
        "limit": limit
    }
    
    resp = requests.get(url, headers=headers, params=params)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"[OpenSea NFT Fetch] Error {resp.status_code} for '{collection_slug}': {resp.text}")
        return {}


def store_opensea_nfts(db_path, table_name, slug_name, nfts_data):
    """
    Inserts each NFT from nfts_data["nfts"] into opensea_nfts, linking them
    via 'slug_name'. We do NOT store the 'collection' field from the JSON,
    but instead rely on 'slug_name' to identify the parent collection.
    """
    nfts_list = nfts_data.get("nfts", [])
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    insert_sql = f"""
    INSERT INTO {table_name} (
        slug_name,
        identifier,
        contract,
        token_standard,
        name,
        description,
        image_url,
        display_image_url,
        display_animation_url,
        metadata_url,
        opensea_url,
        updated_at,
        is_disabled,
        is_nsfw,
        fetched_at
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, datetime('now'))
    """
    
    for nft in nfts_list:
        identifier = nft.get("identifier", "")
        contract = nft.get("contract", "")
        token_standard = nft.get("token_standard", "")
        name = nft.get("name", "")
        desc = nft.get("description", "")
        image_url = nft.get("image_url", "")
        display_image = nft.get("display_image_url", "")
        display_animation = nft.get("display_animation_url", "")
        metadata_url = nft.get("metadata_url", "")
        opensea_url = nft.get("opensea_url", "")
        updated_at = nft.get("updated_at", "")
        
        # Convert booleans to int (1/0)
        is_disabled = 1 if nft.get("is_disabled", False) else 0
        is_nsfw = 1 if nft.get("is_nsfw", False) else 0
        
        c.execute(insert_sql, (
            slug_name,
            identifier,
            contract,
            token_standard,
            name,
            desc,
            image_url,
            display_image,
            display_animation,
            metadata_url,
            opensea_url,
            updated_at,
            is_disabled,
            is_nsfw
        ))
    
    conn.commit()
    conn.close()


def fetch_opensea_nfts_for_collections(db_path="sampling_frame.db", table_collections="sampled_collections",
                                       table_nfts="opensea_nfts", api_key="YOUR_API_KEY", limit=10):
    """
    1) Ensure 'opensea_nfts' table exists.
    2) Query 'sampled_collections' for marketplace='OpenSea', using 'slug_name'
    3) For each, fetch up to {limit} items from OpenSea.
    4) Store them in 'opensea_nfts'.
    """
    # Create table if not exists
    create_opensea_nfts_table(db_path, table_nfts)
    
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Query the relevant rows
    c.execute(f"""
    SELECT slug_name
    FROM {table_collections}
    WHERE marketplace='OpenSea'
      AND slug_name IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    for idx, (slug,) in enumerate(rows, start=1):
        print(f"[{idx}/{len(rows)}] Fetching up to {limit} NFTs for slug='{slug}'")
        nfts_data = fetch_opensea_nfts(slug, api_key, limit=limit)
        store_opensea_nfts(db_path, table_nfts, slug, nfts_data)
    
    print("Done fetching NFT data for all OpenSea collections in sampling_frame.db.")
    

In [5]:
if __name__ == "__main__":
    OPENSEA_API_KEY = "32454cc75657485c8306c86fa37141ec"
    fetch_opensea_nfts_for_collections(
        db_path="sampling_frame.db",
        table_collections="sampled_collections",
        table_nfts="opensea_nfts",
        api_key=OPENSEA_API_KEY,
        limit=10
    )

[1/76] Fetching up to 10 NFTs for slug='zora-posts-19156'
[2/76] Fetching up to 10 NFTs for slug='titanshade'
[3/76] Fetching up to 10 NFTs for slug='summer-vibe-10'
[4/76] Fetching up to 10 NFTs for slug='stained-glasswindows'
[5/76] Fetching up to 10 NFTs for slug='shib-chadmoney'
[6/76] Fetching up to 10 NFTs for slug='me-physical-collectibles-10'
[7/76] Fetching up to 10 NFTs for slug='lens-collect-profile-199929-publication-3'
[8/76] Fetching up to 10 NFTs for slug='hell-angels'
[9/76] Fetching up to 10 NFTs for slug='glow-panda-genesis'
[10/76] Fetching up to 10 NFTs for slug='giants-g-nine-2024-emblem'
[11/76] Fetching up to 10 NFTs for slug='dump-pepe-5'
[12/76] Fetching up to 10 NFTs for slug='delfin-ohno-blast'
[13/76] Fetching up to 10 NFTs for slug='chumpzgoded-season-one'
[14/76] Fetching up to 10 NFTs for slug='bbbdf-e'
[15/76] Fetching up to 10 NFTs for slug='zora-posts-21385'
[16/76] Fetching up to 10 NFTs for slug='municipios-del-futuro-cluster-de-iot-de-la-comunid'
[1

Rarible

In [6]:
def create_rarible_nfts_table(db_path="sample_1.db", table_name="rarible_nfts"):
    """
    Creates a table 'rarible_nfts' with columns similar to 'opensea_nfts',
    storing key fields from the Rarible item-level data plus a link back 
    to 'collection_id' in 'sampled_collections'.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    c.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        -- references the original 'id:blockchain' format from 'sampled_collections'
        collection_id TEXT,
        
        item_id TEXT,          -- item["id"] e.g. "ETHEREUM:0xb66a6...:123"
        blockchain TEXT,       -- item["blockchain"]
        contract TEXT,         -- item["contract"]
        token_id TEXT,         -- item["tokenId"]
        
        name TEXT,             -- from item["meta"]["name"]
        description TEXT,      -- from item["meta"]["description"]
        image_url TEXT,        -- optional, if we parse item["meta"]["content"] for an image
        minted_at TEXT,        -- item["mintedAt"]
        last_updated TEXT,     -- item["lastUpdatedAt"]
        supply REAL,           -- item["supply"] 
        owner_if_single TEXT,  -- item["ownerIfSingle"] if we want
        
        project_url TEXT,      -- e.g. item["meta"]["externalUri"] if present
        created_at TEXT,       -- item["meta"]["createdAt"]
        
        fetched_at TEXT
    )
    """)
    
    conn.commit()
    conn.close()

In [7]:
def transform_collection_id_for_rarible(orig_id):
    """
    Convert 'id:blockchain' => 'BLOCKCHAIN:0xid' for use in Rarible API.
    e.g. '0x5948abcd:ethereum' => 'ETHEREUM:0x5948abcd'
    """
    if ":" not in orig_id:
        return None
    coll_id, chain = orig_id.split(":")
    return f"{chain.upper()}:{coll_id}"

def fetch_rarible_nfts(collection_param, api_key, size=10):
    """
    Calls Rarible endpoint:
      GET /v0.1/items/byCollection?collection={collection_param}&size={size}
    e.g. collection_param = 'ETHEREUM:0xb66a603f...'
    Returns a dict with keys 'continuation' and 'items' (list).
    """
    url = "https://api.rarible.org/v0.1/items/byCollection"
    headers = {
        "accept": "application/json",
        "X-API-KEY": api_key
    }
    params = {
        "collection": collection_param,
        "size": size
    }
    resp = requests.get(url, headers=headers, params=params)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"[Rarible NFT Fetch] Error {resp.status_code} => {resp.text}")
        return {}

def store_rarible_nfts(db_path, table_name, orig_id, items_list):
    """
    Insert up to 10 items into 'rarible_nfts' table, storing relevant fields.
    :param orig_id: The original 'id:blockchain' used in 'sampled_collections'
    :param items_list: list of item dicts from the Rarible API
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    insert_sql = f"""
    INSERT INTO {table_name} (
        collection_id,
        item_id,
        blockchain,
        contract,
        token_id,
        name,
        description,
        image_url,
        minted_at,
        last_updated,
        supply,
        owner_if_single,
        project_url,
        created_at,
        fetched_at
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, datetime('now'))
    """
    
    for item in items_list:
        item_id = item.get("id", "")
        blockchain = item.get("blockchain", "")
        contract = item.get("contract", "")
        token_id = str(item.get("tokenId", ""))
        supply = item.get("supply", 0)
        minted_at = item.get("mintedAt", "")
        last_updated = item.get("lastUpdatedAt", "")
        owner_if_single = item.get("ownerIfSingle", "")
        
        meta = item.get("meta", {})
        name = meta.get("name", "")
        description = meta.get("description", "")
        created_at = meta.get("createdAt", "")
        project_url = meta.get("externalUri", "")
        
        # Optional: parse meta["content"] for image
        # The example doesn't show a direct "url", so we may store 'N/A' or parse further
        content_list = meta.get("content", [])
        image_url = "N/A"
        for content_obj in content_list:
            # if there's a recognized URL, parse it
            # The example lacks a 'url' field, so we skip. 
            # If your real data has it, you might do:
            # image_url = content_obj.get("url", "N/A")
            # break
            pass
        
        c.execute(insert_sql, (
            orig_id,      # collection_id (as it appears in 'sampled_collections')
            item_id,
            blockchain,
            contract,
            token_id,
            name,
            description,
            image_url,
            minted_at,
            last_updated,
            supply,
            owner_if_single,
            project_url,
            created_at
        ))
    
    conn.commit()
    conn.close()

def fetch_rarible_nfts_for_collections(db_path="sample_1.db", table_collections="sampled_collections",
                                       table_nfts="rarible_nfts", api_key="YOUR_RARIBLE_API_KEY", size=10):
    """
    1) Create the 'rarible_nfts' table if not exists.
    2) Query 'sampled_collections' where marketplace='Rarible'.
    3) Transform 'collection_id:blockchain' to 'BLOCKCHAIN:collection_id'.
    4) Fetch items (size=10) from Rarible's endpoint.
    5) Store them in 'rarible_nfts'.
    """
    # 1) create table
    create_rarible_nfts_table(db_path, table_nfts)
    
    # 2) query the relevant rows
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute(f"""
    SELECT collection_id
    FROM {table_collections}
    WHERE marketplace='Rarible'
      AND collection_id IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    count = 0
    for (orig_id,) in rows:
        rarible_format = transform_collection_id_for_rarible(orig_id)
        if not rarible_format:
            print(f"Skipping invalid format => {orig_id}")
            continue
        
        data = fetch_rarible_nfts(rarible_format, api_key, size=size)
        items = data.get("items", [])
        
        store_rarible_nfts(db_path, table_nfts, orig_id, items)
        count += 1
        print(f"[{count}/{len(rows)}] Inserted up to {len(items)} items for {orig_id}")
    
    print("Done fetching and storing Rarible NFTs in", table_nfts)

In [8]:
if __name__ == "__main__":
    RARIBLE_API_KEY = "1c4706e0-6fe2-4181-bb4d-70322cb866b0"
    fetch_rarible_nfts_for_collections(
        db_path="sampling_frame.db",
        table_collections="sampled_collections",
        table_nfts="rarible_nfts",
        api_key=RARIBLE_API_KEY,
        size=10
    )

[1/112] Inserted up to 1 items for 0xb2ffa6075e0594cd62a2035381466cbf2f022274:zksync
[2/112] Inserted up to 1 items for 0x8009c4e95f80cbf59127edafdeaa20eb7abf3563:base
[3/112] Inserted up to 1 items for 0x482f1759dd48df2672c0b9a5fc6791b19f4fb7d3:etherlink
[4/112] Inserted up to 10 items for 0x358f263b33c338719f4fa98ecc807a82228a902f:match
[5/112] Inserted up to 1 items for 0xc219d80cac121b59c232041f72819d21e12e223ce8c543a7609a177b6fd16a0f:aptos
[6/112] Inserted up to 10 items for 0x716ad1b6222046289c1664825cd9e4caf6253aec:match
[7/112] Inserted up to 10 items for 0x1d3fb02ec926043c39f7b4b0bade786f36472471:base
[8/112] Inserted up to 0 items for 0x55b1316d3065f4f88733d0aed9632a5ab8906ebd:base
[9/112] Inserted up to 1 items for 0x77fef3c4c258da9d2a6ce739fa5436825ec28bb4:arbitrum
[10/112] Inserted up to 4 items for 0x14021c550e81fccc20a3b37fceab9b5461dde99b:base
[11/112] Inserted up to 0 items for 0xfe758872fe3a1c91b226dc280a4660850409e10a:astarzkevm
[12/112] Inserted up to 10 items for 0

MagicEden

In [9]:
def create_magiceden_nfts_table(db_path="sample_1.db", table_name="magiceden_nfts"):
    """
    Creates a table 'magiceden_nfts' with columns for relevant fields from
    the Magic Eden listing JSON, plus a link back to 'sampled_collections' via slug_name.
    
    We'll have 27 columns total:
      1) id (PK)
      2) collection_slug
      ...
      26) token_properties_json
      27) fetched_at
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Below we define 27 columns total
    c.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        
        collection_slug TEXT,

        pdaAddress TEXT,
        auctionHouse TEXT,
        tokenAddress TEXT,
        tokenMint TEXT,
        seller TEXT,
        sellerReferral TEXT,
        tokenSize REAL,
        price REAL,
        expiry REAL,
        
        rarity_json TEXT,
        extra_json TEXT,
        listingSource TEXT,
        
        token_mintAddress TEXT,
        token_owner TEXT,
        token_supply REAL,
        token_collection TEXT,
        token_name TEXT,
        token_updateAuthority TEXT,
        token_primarySaleHappened INTEGER,
        token_sellerFeeBasisPoints REAL,
        token_image TEXT,
        token_animationUrl TEXT,
        token_externalUrl TEXT,
        
        token_attributes_json TEXT,
        token_properties_json TEXT,
        
        fetched_at TEXT
    )
    """)
    
    conn.commit()
    conn.close()

def fetch_magiceden_listings(collection_slug, limit=10):
    """
    GET https://api-mainnet.magiceden.dev/v2/collections/{collection_slug}/listings?limit={limit}
    Returns a list of dicts if successful, else empty list.
    """
    base_url = "https://api-mainnet.magiceden.dev/v2/collections"
    url = f"{base_url}/{collection_slug}/listings"
    params = {"limit": limit}
    headers = {"accept": "application/json"}
    
    resp = requests.get(url, headers=headers, params=params)
    if resp.status_code == 200:
        return resp.json()  # Should be a list of listing dicts
    else:
        print(f"[MagicEden NFT Fetch] Error {resp.status_code} for slug='{collection_slug}': {resp.text}")
        return []

def store_magiceden_nfts(db_path, table_name, slug_name, listings):
    """
    Insert each listing item into 'magiceden_nfts'.
    We must provide EXACTLY 26 placeholders for these columns (excluding 'id' + including 'fetched_at' as a datetime call).
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # 27 columns total => 'id' is autoincrement, so we have 26 placeholders
    insert_sql = f"""
    INSERT INTO {table_name} (
        collection_slug,
        
        pdaAddress,
        auctionHouse,
        tokenAddress,
        tokenMint,
        seller,
        sellerReferral,
        tokenSize,
        price,
        expiry,
        
        rarity_json,
        extra_json,
        listingSource,
        
        token_mintAddress,
        token_owner,
        token_supply,
        token_collection,
        token_name,
        token_updateAuthority,
        token_primarySaleHappened,
        token_sellerFeeBasisPoints,
        token_image,
        token_animationUrl,
        token_externalUrl,
        
        token_attributes_json,
        token_properties_json,
        
        fetched_at
    )
    VALUES (
        ?,  -- collection_slug
        ?,  -- pdaAddress
        ?,  -- auctionHouse
        ?,  -- tokenAddress
        ?,  -- tokenMint
        ?,  -- seller
        ?,  -- sellerReferral
        ?,  -- tokenSize
        ?,  -- price
        ?,  -- expiry
        ?,  -- rarity_json
        ?,  -- extra_json
        ?,  -- listingSource
        ?,  -- token_mintAddress
        ?,  -- token_owner
        ?,  -- token_supply
        ?,  -- token_collection
        ?,  -- token_name
        ?,  -- token_updateAuthority
        ?,  -- token_primarySaleHappened
        ?,  -- token_sellerFeeBasisPoints
        ?,  -- token_image
        ?,  -- token_animationUrl
        ?,  -- token_externalUrl
        ?,  -- token_attributes_json
        ?,  -- token_properties_json
        datetime('now')  -- fetched_at
    )
    """
    # Notice we have exactly 26 placeholders (the last column uses datetime('now')).

    for listing in listings:
        pdaAddress = listing.get("pdaAddress", "")
        auctionHouse = listing.get("auctionHouse", "")
        tokenAddress = listing.get("tokenAddress", "")
        tokenMint = listing.get("tokenMint", "")
        seller = listing.get("seller", "")
        sellerReferral = listing.get("sellerReferral", "")
        tokenSize = listing.get("tokenSize", 0)
        price = listing.get("price", 0)
        expiry = listing.get("expiry", 0)
        
        # Convert 'rarity' and 'extra' to JSON strings
        rarity_json = json.dumps(listing.get("rarity", {}))
        extra_json = json.dumps(listing.get("extra", {}))
        
        listingSource = listing.get("listingSource", "")
        
        token_obj = listing.get("token", {})
        token_mintAddress = token_obj.get("mintAddress", "")
        token_owner = token_obj.get("owner", "")
        token_supply = token_obj.get("supply", 0)
        token_collection = token_obj.get("collection", "")
        token_name = token_obj.get("name", "")
        token_updateAuthority = token_obj.get("updateAuthority", "")
        
        primarySaleHappened = 1 if token_obj.get("primarySaleHappened", False) else 0
        sellerFeeBasisPoints = token_obj.get("sellerFeeBasisPoints", 0)
        token_image = token_obj.get("image", "")
        token_animationUrl = token_obj.get("animationUrl", "")
        token_externalUrl = token_obj.get("externalUrl", "")
        
        attributes_json = json.dumps(token_obj.get("attributes", []))
        properties_json = json.dumps(token_obj.get("properties", {}))
        
        c.execute(insert_sql, (
            slug_name,
            pdaAddress,
            auctionHouse,
            tokenAddress,
            tokenMint,
            seller,
            sellerReferral,
            tokenSize,
            price,
            expiry,
            rarity_json,
            extra_json,
            listingSource,
            token_mintAddress,
            token_owner,
            token_supply,
            token_collection,
            token_name,
            token_updateAuthority,
            primarySaleHappened,
            sellerFeeBasisPoints,
            token_image,
            token_animationUrl,
            token_externalUrl,
            attributes_json,
            properties_json
            # fetched_at => datetime('now') in SQL
        ))
    
    conn.commit()
    conn.close()

def fetch_magiceden_nfts_for_collections(db_path="sample_1.db",
                                         table_collections="sampled_collections",
                                         table_nfts="magiceden_nfts",
                                         limit=10):
    """
    Creates table if necessary, then for each row with marketplace='MagicEden',
    uses 'slug_name' to fetch up to {limit} items from Magic Eden,
    storing them in magiceden_nfts.
    """
    create_magiceden_nfts_table(db_path, table_nfts)
    
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute(f"""
    SELECT slug_name
    FROM {table_collections}
    WHERE marketplace='MagicEden'
      AND slug_name IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    count = 0
    for (slug_name,) in rows:
        data = fetch_magiceden_listings(slug_name, limit=limit)
        store_magiceden_nfts(db_path, table_nfts, slug_name, data)
        count += 1
        print(f"[{count}/{len(rows)}] Inserted up to {len(data)} items for MagicEden => {slug_name}")
    
    print(f"Done fetching/storing MagicEden listings in {table_nfts}")

In [10]:
if __name__ == "__main__":
    fetch_magiceden_nfts_for_collections(
        db_path="sampling_frame.db",
        table_collections="sampled_collections",
        table_nfts="magiceden_nfts",
        limit=10
    )

[1/50] Inserted up to 10 items for MagicEden => world_cup_frog
[2/50] Inserted up to 10 items for MagicEden => synergy_land
[3/50] Inserted up to 10 items for MagicEden => solshades_vouchers
[4/50] Inserted up to 10 items for MagicEden => salt_bandits
[5/50] Inserted up to 0 items for MagicEden => p2p
[6/50] Inserted up to 10 items for MagicEden => monkelines
[7/50] Inserted up to 0 items for MagicEden => justcoffee
[8/50] Inserted up to 10 items for MagicEden => bunny_state
[9/50] Inserted up to 0 items for MagicEden => blue_gold_collection
[10/50] Inserted up to 10 items for MagicEden => bit_monkey_club
[11/50] Inserted up to 10 items for MagicEden => yetiz
[12/50] Inserted up to 7 items for MagicEden => swirlogy
[13/50] Inserted up to 10 items for MagicEden => islesofmeta
[14/50] Inserted up to 5 items for MagicEden => dramwolf
[15/50] Inserted up to 10 items for MagicEden => abc_marvel
[16/50] Inserted up to 7 items for MagicEden => spotted_lanternflies
[17/50] Inserted up to 10 it

In [13]:
def add_new_columns_if_not_exists(db_path="sampling_frame.db", table_name="sampled_collections"):
    """
    Adds the column 'total_supply' to the 'sampled_collections' table
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    cols_to_add = [
        ("total_supply", "REAL")
    ]
    
    for col, col_type in cols_to_add:
        alter_stmt = f"ALTER TABLE {table_name} ADD COLUMN {col} {col_type}"
        try:
            c.execute(alter_stmt)
            print(f"Added column '{col}' to {table_name}")
        except sqlite3.OperationalError as e:
            # Likely column already exists
            pass
    
    conn.commit()
    conn.close()

In [11]:
def fetch_atomic_collection_stats(collection_name):
    """
    Calls /atomicassets/v1/collections/{collection_name}/stats"
    """
    url = f"https://wax.api.atomicassets.io/atomicassets/v1/collections/{collection_name}/stats"
    resp = requests.get(url)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"[Atomic Stats] Error {resp.status_code} for collection='{collection_name}': {resp.text}")
        return {}

def update_atomic_collection_total_supply(db_path="sample_1.db",
                                          table_name="sampled_collections"):
    """
    1) Query 'sampled_collections' where marketplace='Atomic',
       using 'slug_name' as the collection_name.
    2) For each row, call fetch_atomic_collection_stats(...),
       parse data["data"]["assets"], and store it in 'total_supply'.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    c.execute(f"""
    SELECT slug_name
    FROM {table_name}
    WHERE marketplace='Atomic'
      AND slug_name IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    print(f"Found {len(rows)} 'Atomic' collections to update total_supply.")
    
    count = 0
    for (collection_name,) in rows:
        stats_data = fetch_atomic_collection_stats(collection_name)
        # stats_data => { "success": bool, "data": {...}, ... }
        if not stats_data.get("success", False):
            print(f"Skipping {collection_name}, 'success' is False or missing.")
            continue
        
        # If "assets" is None or missing, default to "0"
        assets_str = stats_data.get("data", {}).get("assets")
        if assets_str is None:
            assets_str = "0"
        
        try:
            assets_count = int(assets_str)
        except ValueError:
            assets_count = 0  # or handle differently if needed
        
        conn2 = sqlite3.connect(db_path)
        c2 = conn2.cursor()
        update_stmt = f"""
        UPDATE {table_name}
        SET total_supply = ?
        WHERE marketplace='Atomic'
          AND slug_name=?
        """
        c2.execute(update_stmt, (assets_count, collection_name))
        conn2.commit()
        conn2.close()
        
        count += 1
        print(f"[{count}/{len(rows)}] Updated total_supply={assets_count} for {collection_name}")
    
    print("Done updating total_supply for Atomic collections.")

In [14]:
if __name__ == "__main__":
    add_new_columns_if_not_exists(
        db_path="sampling_frame.db",
        table_name="sampled_collections"
        )

    update_atomic_collection_total_supply(
        db_path="sampling_frame.db",
        table_name="sampled_collections"
    )

Added column 'total_supply' to sampled_collections
Found 20 'Atomic' collections to update total_supply.
[1/20] Updated total_supply=30 for dimebagswaxd
[2/20] Updated total_supply=5 for alienwor1dsz
[3/20] Updated total_supply=6 for watercolorzz
[4/20] Updated total_supply=1 for adidasshoes3
[5/20] Updated total_supply=15246 for varialandsio
[6/20] Updated total_supply=22 for waxwalletsto
[7/20] Updated total_supply=3 for ycjhajx5bcol
[8/20] Updated total_supply=2 for vikez1modern
[9/20] Updated total_supply=10 for nftartdesing
[10/20] Updated total_supply=5 for bestpepeever
[11/20] Updated total_supply=1 for agoratest123
[12/20] Updated total_supply=22 for oddsandendss
[13/20] Updated total_supply=24 for agruneforyou
[14/20] Updated total_supply=5 for 1a2b3c4d5e11
[15/20] Updated total_supply=20 for targetedby11
[16/20] Updated total_supply=0 for ondjlfgjbuzp
[17/20] Updated total_supply=10 for dreamhunters
[18/20] Updated total_supply=20 for cardinallind
[19/20] Updated total_supply

In [15]:
def create_atomic_nfts_table(db_path="sample_1.db", table_name="atomic_nfts"):
    """
    Creates a table that stores item-level data from the AtomicAssets API.
    Adjust columns as needed to match your chosen fields.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    c.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        
        -- references the original 'collection_name' from 'sampled_collections' (Atomic)
        collection_name TEXT,
        
        asset_id TEXT,
        owner TEXT,
        item_name TEXT,
        is_transferable INTEGER,
        is_burnable INTEGER,
        
        template_id TEXT,        -- from template["template_id"]
        max_supply TEXT,         -- from template["max_supply"]
        issued_supply TEXT,      -- from template["issued_supply"]
        
        minted_at_time TEXT,     -- item["minted_at_time"]
        minted_at_block TEXT,    -- item["minted_at_block"]
        
        data_json TEXT,          -- optionally store the entire item if you like
        fetched_at TEXT
    )
    """)
    
    conn.commit()
    conn.close()

In [16]:
def fetch_atomic_assets(collection_name, limit=10):
    """
    Calls the endpoint:
      GET /atomicassets/v1/assets?collection_name={collection_name}&limit={limit}
    Returns a dict with "success", "data" (list), and "query_time".
    We'll return the entire JSON for reference.
    """
    base_url = "https://wax.api.atomicassets.io/atomicassets/v1/assets"
    params = {
        "collection_name": collection_name,
        "limit": limit
    }
    resp = requests.get(base_url, params=params)
    if resp.status_code == 200:
        return resp.json()  # e.g. { "success": true, "data": [...], "query_time": 0 }
    else:
        print(f"[Atomic NFT Fetch] Error {resp.status_code} => {resp.text}")
        return {}

def store_atomic_nfts(db_path, table_name, coll_name, items):
    """
    Inserts the provided item-level data into 'atomic_nfts'.
    items is a list of dicts from data["data"] in the AtomicAssets response.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    insert_sql = f"""
    INSERT INTO {table_name} (
        collection_name,
        asset_id,
        owner,
        item_name,
        is_transferable,
        is_burnable,
        template_id,
        max_supply,
        issued_supply,
        minted_at_time,
        minted_at_block,
        data_json,
        fetched_at
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, datetime('now'))
    """
    
    for item in items:
        asset_id = item.get("asset_id", "")
        owner = item.get("owner", "")
        item_name = item.get("name", "")
        
        # Convert booleans to int
        is_transferable = 1 if item.get("is_transferable", False) else 0
        is_burnable = 1 if item.get("is_burnable", False) else 0
        
        # Safely handle template, which can be None
        template = item.get("template") or {}
        template_id = template.get("template_id", "")
        max_supply = template.get("max_supply", "")
        issued_supply = template.get("issued_supply", "")
        
        minted_at_time = item.get("minted_at_time", "")
        minted_at_block = item.get("minted_at_block", "")
        
        data_json = json.dumps(item)
        
        c.execute(insert_sql, (
            coll_name,
            asset_id,
            owner,
            item_name,
            is_transferable,
            is_burnable,
            template_id,
            max_supply,
            issued_supply,
            minted_at_time,
            minted_at_block,
            data_json
        ))
    
    conn.commit()
    conn.close()
    
def fetch_atomic_nfts_for_collections(db_path="sample_1.db",
                                      table_collections="sampled_collections",
                                      table_nfts="atomic_nfts",
                                      limit=10):
    """
    1) Create the 'atomic_nfts' table if not exists.
    2) Query 'sampled_collections' for marketplace='Atomic'.
       We'll assume there's a column 'collection_name' for each row.
    3) For each row, call fetch_atomic_assets(collection_name, limit={limit}),
       then store them in 'atomic_nfts'.
    """
    # 1) create table
    create_atomic_nfts_table(db_path, table_nfts)
    
    # 2) get the relevant rows from 'sampled_collections'
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute(f"""
    SELECT slug_name
    FROM {table_collections}
    WHERE marketplace='Atomic'
      AND slug_name IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    print(f"Found {len(rows)} Atomic collections to fetch up to {limit} assets each.")
    
    count = 0
    for (coll_name,) in rows:
        print(f"[{count+1}/{len(rows)}] Fetching {limit} assets for collection_name='{coll_name}'")
        
        data = fetch_atomic_assets(coll_name, limit=limit)
        items = data.get("data", [])  # a list of item dicts
        
        store_atomic_nfts(db_path, table_nfts, coll_name, items)
        count += 1
    
    print("Done fetching/storing Atomic NFTs in", table_nfts)

In [17]:
if __name__ == "__main__":
    fetch_atomic_nfts_for_collections(
        db_path="sampling_frame.db",
        table_collections="sampled_collections",
        table_nfts="atomic_nfts",
        limit=10
    )

Found 20 Atomic collections to fetch up to 10 assets each.
[1/20] Fetching 10 assets for collection_name='dimebagswaxd'
[2/20] Fetching 10 assets for collection_name='alienwor1dsz'
[3/20] Fetching 10 assets for collection_name='watercolorzz'
[4/20] Fetching 10 assets for collection_name='adidasshoes3'
[5/20] Fetching 10 assets for collection_name='varialandsio'
[6/20] Fetching 10 assets for collection_name='waxwalletsto'
[7/20] Fetching 10 assets for collection_name='ycjhajx5bcol'
[8/20] Fetching 10 assets for collection_name='vikez1modern'
[9/20] Fetching 10 assets for collection_name='nftartdesing'
[10/20] Fetching 10 assets for collection_name='bestpepeever'
[11/20] Fetching 10 assets for collection_name='agoratest123'
[12/20] Fetching 10 assets for collection_name='oddsandendss'
[13/20] Fetching 10 assets for collection_name='agruneforyou'
[14/20] Fetching 10 assets for collection_name='1a2b3c4d5e11'
[15/20] Fetching 10 assets for collection_name='targetedby11'
[16/20] Fetching 10 